# Real effective exchange rates 

In [70]:
import numpy as np
import pandas as pd
import os
import re
import duckdb

## Compute REER weights

In [71]:
# inputfolder = 'ADB-MRIO'
# outputroot = 'reer-weights'

inputfolder = 'ADB-MRIO62'
outputroot = 'reer62-weights'

filelist = [file for file in os.listdir(f'../data/interim/{inputfolder}') if not file.startswith('.')]
filelist.sort()

sectors = pd.read_excel('../data/raw/sectors.xlsx')
sectors = sectors.drop_duplicates(subset='ind', ignore_index=True)

# G = 73      # Number of countries + ROW
G = 63      
N = 35      # Number of sectors
f = 5       # Number of final demand components

np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

In [72]:
def to_df(matrix, year, method=None):
    df = pd.DataFrame(matrix)
    df.insert(0, 't', year)
    if method is not None:
        df.insert(0, 'method', method)
    return df

In [73]:
DF = pd.DataFrame()
DF_sector = pd.DataFrame()

for file in filelist:
    
    year = re.search('[0-9]{4}', file).group()

    mrio = duckdb.sql(
        f"""
        SELECT * EXCLUDE(C0)
        FROM read_parquet('../data/interim/{inputfolder}/{file}')
        """
    ).df()
    mrio = mrio.values

    x = mrio[-1][:(G*N)]
    Z = mrio[:(G*N)][:, :(G*N)]
    va = np.sum(mrio[-7:-1][:, :(G*N)], axis=0)
    Y_big = mrio[:(G*N)][:, (G*N):-1]
    Y = Y_big @ np.kron(np.eye(G), np.ones((f, 1)))
    y = np.sum(Y, axis=0)
    v = np.where(x != 0, va/x, 0)
    Dx = np.diag(np.where(x != 0, 1/x, 0))
    A = Z @ Dx
    B = Dx @ Z
    Leontief = np.linalg.inv(np.eye(G*N) - A)
    Ghosh = np.linalg.inv(np.eye(G*N) - B)
    
    agg = np.kron(np.eye(G), np.ones((N, 1)))
    Z_agg = np.transpose(agg) @ Z @ agg
    Y_agg = np.transpose(agg) @ Y
    va_agg = va @ agg
    D_agg = Z_agg + Y_agg

    z_agg = np.sum(Z_agg, axis=0)
    d_agg = np.sum(D_agg, axis=0)
    x_agg = z_agg + va_agg
    e_agg = z_agg + y

    A_agg = Z_agg @ np.diag(1/x_agg)
    B_agg = np.diag(1/x_agg) @ Z_agg
    Leontief_agg = np.linalg.inv(np.eye(G) - A_agg)
    Ghosh_agg = np.linalg.inv(np.eye(G) - B_agg)

    # Weights matrices: Conventional

    S = np.diag(1/d_agg) @ np.transpose(D_agg)
    W = np.diag(1/e_agg) @ D_agg
    SW0 = S @ W * (1 - np.eye(G))
    Weights = np.eye(G) - np.diag(1/np.sum(SW0, axis=1)) @ SW0

    # Weights matrices: Bems & Johnson

    Sy = np.diag(1/x_agg) @ Y_agg
    Sz = np.diag(1/x_agg) @ Z_agg
    Wy = np.diag(1/y) @ np.transpose(Y_agg)
    Wz = np.diag(1/z_agg) @ np.transpose(Z_agg)
    v_agg = np.diag(va_agg/x_agg)
    SW0_BJ = Ghosh_agg @ Sy @ Wy @ np.transpose(Leontief_agg) @ v_agg * (1-np.eye(G))
    Weights_BJ = np.eye(G) - np.diag(1/np.sum(SW0_BJ, axis=1)) @ SW0_BJ

    # Weights matrices: Patel, Wang, & Wei

    SSy = Dx @ Y
    WWy = np.diag(1/y) @ np.transpose(Y)
    SW0_PWW = Ghosh @ SSy @ WWy @ np.transpose(Leontief) @ np.diag(v) * (1 - np.eye(G*N))
    Shares_PWW = np.diag(1/np.sum(SW0_PWW, axis=1))
    Weights_PWW = np.eye(G*N) - Shares_PWW @ SW0_PWW
    Weights_PWW[np.isnan(Weights_PWW)] = 0

    R = np.diag(1/va_agg) @ np.kron(np.ones((G, 1)), va) * np.transpose(np.kron(np.eye(G), np.ones((N, 1))))
    SW0_PWW_agg = R @ (Ghosh @ SSy @ WWy @ np.transpose(Leontief) @ np.diag(v)) @ agg * (1 - np.eye(G))
    Shares_PWW_agg = np.diag(1/np.sum(SW0_PWW_agg, axis=1))
    Weights_PWW_agg = np.eye(G) - Shares_PWW_agg @ SW0_PWW_agg

    # Append
    
    DF = pd.concat(
        [
            DF,
            to_df(Weights, year, 'conventional'), 
            to_df(Weights_BJ, year, 'bj'), 
            to_df(Weights_PWW_agg, year, 'pww')
        ],
        ignore_index=True
    )
    DF_sector = pd.concat([DF_sector, to_df(Weights_PWW, year)], ignore_index=True)

    print(f'{year} done')

DF.to_parquet(f'../data/interim/reer/{outputroot}.parquet', index=False)
DF_sector.to_parquet(f'../data/interim/reer/{outputroot}-sector.parquet', index=False)

2000 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
2020 done
2021 done


In [74]:
DF

,method,t,0,1,2,3,4,5,6,7,...,53,54,55,56,57,58,59,60,61,62
0,conventional,2000,1.000000,-0.002583,-0.005107,-0.000076,-0.006655,-0.017241,-0.006943,-0.053031,...,-0.000126,-0.002003,-0.000011,-0.000010,-0.000137,-0.000056,-0.000124,-0.027796,-0.013527,-0.216143
1,conventional,2000,-0.003381,1.000000,-0.019129,-0.001121,-0.004080,-0.010520,-0.046749,-0.009455,...,-0.000003,-0.000033,-0.000015,-0.000013,-0.000038,-0.000010,-0.000042,-0.002706,-0.001433,-0.081117
2,conventional,2000,-0.003550,-0.010158,1.000000,-0.000663,-0.007343,-0.007029,-0.021462,-0.010124,...,-0.000013,-0.000073,-0.000003,-0.000039,-0.000038,-0.000011,-0.000079,-0.005252,-0.002753,-0.101772
3,conventional,2000,-0.002022,-0.022647,-0.025224,1.000000,-0.008613,-0.004324,-0.012311,-0.011055,...,-0.000011,-0.000073,-0.000023,-0.000186,-0.000053,-0.000059,-0.000087,-0.004180,-0.001157,-0.208141
4,conventional,2000,-0.008481,-0.003972,-0.013463,-0.000415,1.000000,-0.013887,-0.009792,-0.023853,...,-0.000010,-0.000048,-0.000005,-0.000015,-0.000035,-0.000014,-0.000024,-0.005356,-0.005066,-0.290558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3019,pww,2021,-0.014325,-0.004873,-0.006384,-0.000623,-0.005818,-0.011528,-0.017677,-0.094219,...,-0.000817,-0.000785,-0.000047,-0.000259,-0.000305,1.000000,-0.000132,-0.014962,-0.007123,-0.213285
3020,pww,2021,-0.017310,-0.001338,-0.003793,-0.000464,-0.005971,-0.004560,-0.006414,-0.162489,...,-0.000092,-0.000209,-0.000391,-0.000038,-0.000154,-0.000050,1.000000,-0.007807,-0.007640,-0.167706
3021,pww,2021,-0.026876,-0.002994,-0.006903,-0.000581,-0.008285,-0.010478,-0.014855,-0.161922,...,-0.000300,-0.000641,-0.000025,-0.000057,-0.002070,-0.000176,-0.000241,1.000000,-0.015518,-0.145508
3022,pww,2021,-0.014399,-0.001934,-0.006146,-0.000426,-0.012709,-0.011892,-0.010741,-0.320110,...,-0.000260,-0.000332,-0.000024,-0.000040,-0.001100,-0.000147,-0.000415,-0.027283,1.000000,-0.090319


In [75]:
DF_sector

,t,0,1,2,3,4,5,6,7,8,...,2195,2196,2197,2198,2199,2200,2201,2202,2203,2204
0,2000,1.000000,-0.007881,-0.008789,-0.000593,-0.000067,-0.002002,-0.007647,-0.002424,-0.003209,...,-0.000351,-0.001041,-0.002764,-0.002563,-0.003841,-0.002835,-0.002463,-0.001121,-0.001369,-0.000195
1,2000,-0.005391,1.000000,-0.004025,-0.000289,-0.000034,-0.000909,-0.003458,-0.001132,-0.001476,...,-0.000576,-0.002426,-0.005898,-0.007611,-0.007715,-0.008548,-0.006244,-0.003193,-0.003414,-0.000543
2,2000,-0.013326,-0.008922,1.000000,-0.000677,-0.000076,-0.002298,-0.008783,-0.002780,-0.003678,...,-0.000289,-0.000824,-0.002184,-0.001781,-0.003199,-0.001928,-0.001883,-0.000810,-0.001022,-0.000144
3,2000,-0.007231,-0.005144,-0.005440,1.000000,-0.000046,-0.001238,-0.004729,-0.001545,-0.002007,...,-0.000530,-0.002214,-0.005444,-0.006885,-0.007258,-0.007709,-0.005728,-0.002911,-0.003113,-0.000465
4,2000,-0.004965,-0.003661,-0.003720,-0.000278,1.000000,-0.000839,-0.003204,-0.001088,-0.001381,...,-0.000430,-0.001579,-0.004036,-0.004600,-0.005870,-0.005144,-0.003801,-0.001942,-0.002219,-0.000317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35275,2021,-0.000079,-0.001114,-0.000041,-0.000014,-0.000013,-0.000008,-0.000052,-0.000012,-0.000071,...,-0.007245,-0.018229,-0.045586,-0.055307,-0.043678,1.000000,-0.037700,-0.026060,-0.017461,-0.003042
35276,2021,-0.000079,-0.001102,-0.000042,-0.000014,-0.000012,-0.000008,-0.000052,-0.000012,-0.000070,...,-0.007147,-0.017985,-0.044975,-0.054570,-0.043089,-0.053264,1.000000,-0.025713,-0.017228,-0.003001
35277,2021,-0.000077,-0.001079,-0.000041,-0.000013,-0.000012,-0.000008,-0.000051,-0.000012,-0.000069,...,-0.007023,-0.017670,-0.044192,-0.053606,-0.042346,-0.052320,-0.036539,1.000000,-0.016926,-0.002948
35278,2021,-0.000105,-0.001161,-0.000057,-0.000013,-0.000011,-0.000010,-0.000053,-0.000013,-0.000070,...,-0.006300,-0.015793,-0.039570,-0.047834,-0.038000,-0.046667,-0.032588,-0.022531,1.000000,-0.002628


## Compute REER index

In [207]:
# Load deflators
deflators = pd.read_excel(
    '../data/raw/ADB-MRIO-Deflators (for 2016-2021).xlsx',
    sheet_name="Production",
    skiprows=[0,1,3,4],
    header=[0,1]
)
deflators.insert(0, 's', deflators.index+1)
deflators = pd.melt(deflators, id_vars=['s', ('Year', 'Country Code')])
deflators.columns = ['s', 'code', 't', 'i', 'deflator']
deflators['i'] = deflators['i'].str.replace(r'c', '').astype(int)
deflators['t'] = deflators['t'].astype(int)

# Load country-sector value added
va = duckdb.sql(
    """
    SELECT t, s, i, va, 
    FROM read_parquet('../data/final/summary62.parquet')
    """
).df()
va['t'] = va['t'].astype(int)
va_sum = va.groupby(['s', 't'])['va'].sum().reset_index()
va_sum.rename(columns={'va': 'va_sum'}, inplace=True)
va = pd.merge(va, va_sum, on=['s', 't'])
va['va_sh'] = va['va'] / va['va_sum']

deflators = pd.merge(deflators, va)

# Compute aggregate deflator
deflators['def_va_sh'] = deflators['deflator'] * deflators['va_sh']
deflators_agg = deflators.groupby(['t', 's'])['def_va_sh'].sum().reset_index()
deflators_agg.rename(columns={'def_va_sh': 'deflator'}, inplace=True)
deflators_agg['agg'] = 0
deflators_agg['i'] = 0

# Consolidate
deflators['agg'] = 35
deflators.drop(['code', 'va', 'va_sum', 'va_sh', 'def_va_sh'], axis=1, inplace=True)
deflators = pd.concat([deflators, deflators_agg])
deflators = deflators[['t', 'agg', 's', 'i', 'deflator']]
deflators.sort_values(by=['agg', 't', 's', 'i'], inplace=True)

deflators.to_parquet('../data/interim/reer/deflators.parquet', index=False)

In [208]:
deflators

,t,agg,s,i,deflator
0,2000,0,1,0,43.888733
1,2000,0,2,0,59.316288
2,2000,0,3,0,58.907992
3,2000,0,4,0,42.367195
4,2000,0,5,0,42.653496
...,...,...,...,...,...
35027,2021,35,63,31,118.637567
35090,2021,35,63,32,118.637567
35153,2021,35,63,33,118.637567
35216,2021,35,63,34,118.637567


In [240]:
weights = pd.read_parquet('../data/interim/reer/reer62-weights.parquet')
weights['t'] = weights['t'].astype(int)
years = weights['t'].unique()
methods = weights['method'].unique()

In [ ]:
reer = pd.DataFrame()
deflators_agg = deflators[deflators['agg'] == 0]

for method in methods:
    
    weights_m = weights[weights['method'] == method]

    for year in years:
        
        weights_m_t = weights_m[weights['t'] == year].iloc[:, 2:]
        deflators_agg_t = deflators_agg[deflators_agg['t'] == year]['deflator']
        weights_m_t @ deflators_agg_t
        
        


In [242]:
year = 2020
weights_m = weights[weights['method'] == method]
weights_m_t = weights_m[weights['t'] == year]
weights_m_t.iloc[:, 2:]
# weights_m_t

/var/folders/cv/xn1my8q111q1ngz5g62wkt_r0000gn/T/ipykernel_15634/2494845781.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  weights_m_t = weights_m[weights['t'] == year]


,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
2646,1.000000,-0.003000,-0.004526,-0.000363,-0.004786,-0.010462,-0.008733,-0.307734,-0.000343,-0.001549,...,-0.000179,-0.002451,-0.000009,-0.000025,-0.000332,-0.000080,-0.000137,-0.034330,-0.005033,-0.075616
2647,-0.004528,1.000000,-0.016967,-0.004305,-0.003908,-0.006022,-0.054147,-0.056789,-0.000979,-0.028660,...,-0.000041,-0.000027,-0.000013,-0.000045,-0.000162,-0.000036,-0.000066,-0.003924,-0.000567,-0.063231
2648,-0.003847,-0.009555,1.000000,-0.002705,-0.005528,-0.008455,-0.026907,-0.040549,-0.000598,-0.008918,...,-0.000052,-0.000055,-0.000009,-0.000034,-0.000163,-0.000024,-0.000064,-0.008411,-0.002002,-0.069982
2649,-0.003129,-0.024616,-0.027463,1.000000,-0.002863,-0.006723,-0.013275,-0.056029,-0.002229,-0.018656,...,-0.000187,-0.000029,-0.000002,-0.000141,-0.000058,-0.000017,-0.000102,-0.003835,-0.001082,-0.136106
2650,-0.005996,-0.003244,-0.008149,-0.000416,1.000000,-0.016767,-0.012830,-0.225040,-0.000117,-0.001533,...,-0.000019,-0.000029,-0.000004,-0.000018,-0.000159,-0.000029,-0.000118,-0.011491,-0.004106,-0.170015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2704,-0.012297,-0.003648,-0.004427,-0.000304,-0.003536,-0.011205,-0.014637,-0.097178,-0.001112,-0.002190,...,-0.001180,-0.000880,-0.000055,-0.003650,-0.000387,1.000000,-0.000220,-0.037305,-0.007080,-0.252166
2705,-0.005938,-0.001889,-0.003274,-0.000516,-0.004108,-0.011835,-0.010520,-0.150223,-0.000299,-0.000687,...,-0.000061,-0.000171,-0.000427,-0.000024,-0.000325,-0.000062,1.000000,-0.023166,-0.006278,-0.080960
2706,-0.034142,-0.002586,-0.009841,-0.000442,-0.009121,-0.007601,-0.021929,-0.130922,-0.002158,-0.001822,...,-0.000168,-0.001015,-0.000030,-0.000030,-0.001462,-0.000240,-0.000530,1.000000,-0.020795,-0.096882
2707,-0.013359,-0.000996,-0.006251,-0.000333,-0.008698,-0.010509,-0.008967,-0.347156,-0.000529,-0.000934,...,-0.000115,-0.000322,-0.000017,-0.000021,-0.001842,-0.000122,-0.000383,-0.055495,1.000000,-0.071524


In [263]:
deflators_agg = deflators[deflators['agg'] == 0]
weights_m = weights[weights['method'] == method]

year = 2020
weights_m_t = np.array(weights_m[weights['t'] == year].iloc[:, 2:])
deflators_agg_t = deflators_agg[deflators_agg['t'] == year]['deflator']

weights_m_t @ deflators_agg_t

/var/folders/cv/xn1my8q111q1ngz5g62wkt_r0000gn/T/ipykernel_15634/3858800590.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  weights_m_t = np.array(weights_m[weights['t'] == year].iloc[:, 2:])


array([-25.35631682,   0.82254365,  -1.50854448,  20.91315887,
       -45.16410157, -26.98261034,   2.46155658,  26.16559634,
       -14.3363699 ,  -3.62657993,   0.139815  ,  -5.61031883,
       -11.6196069 ,  18.54379983,   0.64225714, -10.3208503 ,
        -1.40387827, -22.71494699, -13.08199911,  -8.00693568,
       -20.30623724, -14.27332306,   8.87123727,  -8.5026546 ,
       -29.13193502,   0.06178351,   8.77943811,   4.65973815,
        10.54933306, -24.69792783,   5.70547946,  -7.31985313,
       -26.25899635, -10.21149739,  -6.15598047,  15.40931177,
       -16.91317206,  -7.37421071,  -5.43215357,  -6.54954091,
       -48.93102388,  -3.61907667,  18.46843688,  39.80661008,
       -21.45797523,   1.83811016,  10.36414715,  15.58875332,
       -21.47483822,  -8.50756779,  -8.69929312,  -7.27492236,
        18.51317662,  12.43828966, -19.06864898,   5.42232251,
        19.90310435,  10.14042483,   9.57053701,  12.98875493,
        -0.64518112,  17.18878479,   0.91896742])

In [254]:
weights_m_t

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
0,1.000000,-0.003000,-0.004526,-0.000363,-0.004786,-0.010462,-0.008733,-0.307734,-0.000343,-0.001549,...,-0.000179,-0.002451,-0.000009,-0.000025,-0.000332,-0.000080,-0.000137,-0.034330,-0.005033,-0.075616
1,-0.004528,1.000000,-0.016967,-0.004305,-0.003908,-0.006022,-0.054147,-0.056789,-0.000979,-0.028660,...,-0.000041,-0.000027,-0.000013,-0.000045,-0.000162,-0.000036,-0.000066,-0.003924,-0.000567,-0.063231
2,-0.003847,-0.009555,1.000000,-0.002705,-0.005528,-0.008455,-0.026907,-0.040549,-0.000598,-0.008918,...,-0.000052,-0.000055,-0.000009,-0.000034,-0.000163,-0.000024,-0.000064,-0.008411,-0.002002,-0.069982
3,-0.003129,-0.024616,-0.027463,1.000000,-0.002863,-0.006723,-0.013275,-0.056029,-0.002229,-0.018656,...,-0.000187,-0.000029,-0.000002,-0.000141,-0.000058,-0.000017,-0.000102,-0.003835,-0.001082,-0.136106
4,-0.005996,-0.003244,-0.008149,-0.000416,1.000000,-0.016767,-0.012830,-0.225040,-0.000117,-0.001533,...,-0.000019,-0.000029,-0.000004,-0.000018,-0.000159,-0.000029,-0.000118,-0.011491,-0.004106,-0.170015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,-0.012297,-0.003648,-0.004427,-0.000304,-0.003536,-0.011205,-0.014637,-0.097178,-0.001112,-0.002190,...,-0.001180,-0.000880,-0.000055,-0.003650,-0.000387,1.000000,-0.000220,-0.037305,-0.007080,-0.252166
59,-0.005938,-0.001889,-0.003274,-0.000516,-0.004108,-0.011835,-0.010520,-0.150223,-0.000299,-0.000687,...,-0.000061,-0.000171,-0.000427,-0.000024,-0.000325,-0.000062,1.000000,-0.023166,-0.006278,-0.080960
60,-0.034142,-0.002586,-0.009841,-0.000442,-0.009121,-0.007601,-0.021929,-0.130922,-0.002158,-0.001822,...,-0.000168,-0.001015,-0.000030,-0.000030,-0.001462,-0.000240,-0.000530,1.000000,-0.020795,-0.096882
61,-0.013359,-0.000996,-0.006251,-0.000333,-0.008698,-0.010509,-0.008967,-0.347156,-0.000529,-0.000934,...,-0.000115,-0.000322,-0.000017,-0.000021,-0.001842,-0.000122,-0.000383,-0.055495,1.000000,-0.071524


In [257]:
deflators_agg_t

0      87.278303
1     105.193910
2     101.370921
3     121.888578
4      67.258741
         ...    
58    117.686827
59    117.864667
60    107.821393
61    130.929405
62    110.422985
Name: deflator, Length: 63, dtype: float64